In [1]:
from mpmath import mp, mpf, findroot, j0, besselj
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

In [10]:
def binomial_like(n, m):
    prod = QQ(1)
    z = QQ(4)*(n**2)
    for k in range(1,m + 1):
        prod *= (z - (2*k - 1)**2)
    return prod / (QQ(2)**(2*m) * (ZZ(m).factorial()))

R = LaurentSeriesRing(RealField(300), 'x',default_prec=300)
x = R.gen()

def Pn_asymptotic(n, y, terms=10):
    # now y = 1/x
    return sum( (-1)**m * binomial_like(n, 2*m) / (QQ(2)**(2*m)) * y**(QQ(2)*m) for m in range(terms) )

def Qn_asymptotic(n, y, terms=10):
    return sum( (-1)**m * binomial_like(n, 2*m + 1) / (QQ(2)**(2*m + 1)) * y**(QQ(2)*m + 1) for m in range(terms) )

P = Pn_asymptotic(0, x, 50)
Q = Qn_asymptotic(0, x, 50)

R_series = (-Q/P)

# alpha is atan(R_series) so we're doing Taylor series atan expansion on R_series

arctan_series_Z = sum([QQ(-1)**k * x**(QQ(2)*k+1) / RealField(700)(RealField(700)(2)*k+1) for k in range(25)])
alpha_series = arctan_series_Z(R_series)

# see the series
# print(alpha_series)

for i in range(alpha_series.prec()):
    if i > 44:
        break
    coeff = alpha_series[i]
    if coeff != 0:
#         print_dyadic256(RealField(300)(f"{coeff}"))
        # if i < 16:
        # print(f"\"{RealField(300)(f"{coeff}")}\",")
        # print_dyadic(RealField(300)(f"{coeff}"))
        # else:
        print_double_double("", RealField(300)(coeff))

(0x0000000000000000, 0x3fc0000000000000),
(0x3c55555555555555, 0xbfb0aaaaaaaaaaab),
(0x3c5999999999999a, 0x3fcad33333333333),
(0xbc92492492492492, 0xbffa358492492492),
(0xbcbc71c71c71c71c, 0x403779a1f8e38e39),
(0xbd0745d1745d1746, 0xc080bd1fc8b1745d),
(0xbd7d89d89d89d89e, 0x40d16b51e66c789e),
(0x3dc5555555555555, 0xc128ecc3af33ab37),
(0x3e2143c3c3c3c3c4, 0x418779dae2b8512f),
(0x3df41e50d79435e5, 0xc1ec296336955c7f),
(0x3ef6dcbaf0618618, 0x4254f5ee683b6432),
(0x3f503a3102cc7a6f, 0xc2c2f51eced6693f),
(0xbfd463c8f5076f81, 0x43347a533f6de60a),
(0x404880bbfc0e1bc0, 0xc3aa0a16e6f97809),
(0xc0cb6bd585e90c96, 0x44234046f37b54fd),
(0x4138221948f1346d, 0xc4a06023472701c9),
(0xc1b32ad2094d60e4, 0x451fc2ca43f8f3cc),
(0xc23c415dbb8800bb, 0xc5a16a9627a7f67c),
(0xc2cb1ab8b093e2ea, 0x462572a8ac1517f7),
(0x434dc2527b496a2e, 0xc6ad7878738aae2b),
(0xc3df4e5acec45b6d, 0x4736768ff362a6a8),
(0xc46f677aa157d669, 0xc7c2e61448ca47b1),


In [14]:
#generate b series
def binomial_like(n, m):
    prod = QQ(1)
    z = QQ(4)*(n**2)
    for k in range(1,m + 1):
        prod *= (z - (2*k - 1)**2)
    return prod / (QQ(2)**(2*m) * (ZZ(m).factorial()))

R = LaurentSeriesRing(RealField(400), 'x',default_prec=400)
x = R.gen()

def Pn_asymptotic(n, y, terms=10):
    # now y = 1/x
    return sum( (-1)**m * binomial_like(n, 2*m) / (QQ(2)**(2*m)) * y**(QQ(2)*m) for m in range(terms) )

def Qn_asymptotic(n, y, terms=10):
    return sum( (-1)**m * binomial_like(n, 2*m + 1) / (QQ(2)**(2*m + 1)) * y**(QQ(2)*m + 1) for m in range(terms) )

P = Pn_asymptotic(0, x, 50)
Q = Qn_asymptotic(0, x, 50)

def sqrt_series(s):
    val = S.valuation()
    lc = S[val]  # Leading coefficient
    b = lc.sqrt() * x**(val // 2)

    for _ in range(5):
        b = (b + S / b) / 2
        b = b
    return b

S = (P**2 + Q**2).truncate(50)

b_series = sqrt_series(S).truncate(30)
#see the series
print(b_series)

print("beta series")
for i in range(24):
    coeff = b_series[i]
    if coeff != 0:
        # print(f"{RealField(300)(f"{coeff}")},")
        # print(double_to_hex(RealField(300)(f"{coeff}")) + ",")
        # print_dyadic(RealField(300)(f"{coeff}"))
        print_double_double("", RealField(300)(f"{coeff}"))


1.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x - 0.0625000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^2 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^3 + 0.103515625000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^4 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^5 - 0.542846679687500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^6 + 0.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000